In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df=pd.read_csv('data.csv',sep=';')
df


,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,06.10.2020,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,27.10.2020,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,03.12.2020,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,12.01.2021,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [ ]:
df.shape
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,2861.0,12.397064,6.084226,1.00,8.0000,14.000,16.00000,22.000
NH4,2858.0,0.758734,2.486247,0.00,0.0800,0.220,0.50000,39.427
BSK5,2860.0,4.316182,2.973997,0.00,2.1600,3.800,5.80000,50.900
Suspended,2845.0,12.931905,16.543097,0.00,6.0000,10.000,15.00000,595.000
O2,2858.0,9.508902,4.428260,0.00,7.0925,8.995,11.52000,90.000
NO3,2860.0,4.316846,6.881188,0.00,1.3900,2.800,5.58250,133.400
NO2,2858.0,0.246128,2.182777,0.00,0.0300,0.059,0.12575,109.000
SO4,2812.0,59.362313,96.582641,0.00,27.0525,37.800,64.64000,3573.400
PO4,2833.0,0.418626,0.771326,0.00,0.1300,0.270,0.47000,13.879
CL,2812.0,93.731991,394.512184,0.02,26.8000,33.900,45.60750,5615.280


In [ ]:
df.isnull().sum()

,0
id,0
date,0
NH4,3
BSK5,1
Suspended,16
O2,3
NO3,1
NO2,3
SO4,49
PO4,28


In [ ]:
df=df.sort_values(by=['id','date'])
df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
43,1,01.12.2010,0.291,3.27,16.7,12.30,59.20,0.090,278.4,0.370,1375.46
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.00
41,1,02.06.2010,0.290,3.52,37.2,7.16,57.87,0.210,268.8,0.240,1120.20
22,1,02.09.2005,0.000,3.68,28.5,14.20,15.51,0.033,422.4,0.900,2063.80
62,1,02.09.2015,0.035,6.28,104.8,7.80,4.68,0.076,694.4,0.530,5615.28


In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month

df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year,month
43,1,2010-12-01,0.291,3.27,16.7,12.30,59.20,0.090,278.4,0.370,1375.46,2010,12
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.00,2001,3
41,1,2010-06-02,0.290,3.52,37.2,7.16,57.87,0.210,268.8,0.240,1120.20,2010,6
22,1,2005-09-02,0.000,3.68,28.5,14.20,15.51,0.033,422.4,0.900,2063.80,2005,9
62,1,2015-09-02,0.035,6.28,104.8,7.80,4.68,0.076,694.4,0.530,5615.28,2015,9


In [ ]:
 df.columns

Index(['id', 'date', 'NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4',
       'PO4', 'CL', 'year', 'month'],
      dtype='object')

In [ ]:
pollutants=['id', 'date', 'NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4',
       'PO4', 'CL']

In [20]:
df=df.dropna(subset=pollutants)
df.isnull().sum()

,0
id,0
date,0
NH4,0
BSK5,0
Suspended,0
O2,0
NO3,0
NO2,0
SO4,0
PO4,0


In [28]:
x=df[['id','year']]
y=df[['NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']]

In [29]:
x_encoded=pd.get_dummies(x,columns=['id'],drop_first=True)


In [30]:
x_train,x_test,y_train,y_test=train_test_split(x_encoded,y,test_size=0.2,random_state=42)

In [31]:
model=MultiOutputRegressor(RandomForestRegressor( n_estimators=100,random_state=42))
model.fit(x_train,y_train)



MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [32]:
y_pred=model.predict(x_test)

In [36]:
print("test data analysis")
for i,pollutant in enumerate(pollutants):
    print(pollutant)
    print(mean_squared_error(y_test.iloc[:,i],y_pred[:,i]))
    print(r2_score(y_test.iloc[:,i],y_pred[:,i]))

test data analysis
id
1.3389030226302183
0.5075698298153315
date
5.4966842778181455
0.2536805521908698
NH4
177.64684047501495
-0.3898093078569824
BSK5
23.804974219550807
-0.024655347555051943
Suspended
14.190326747205722
0.46420823872601225
O2
0.9021202011124738
-2.306102849515763
NO3
799.3361629017251
0.7636185777198815
NO2
0.23319148787932148
0.14388427070369814
SO4
31223.168320328627
0.6324865646628188
PO4


IndexError: single positional indexer is out-of-bounds

In [37]:
station_id=df['id'].unique()
print(station_id)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


In [44]:
station_id=22
year=2024

input_data=pd.DataFrame({'id':[station_id],'year':[year]})
input_data_encoded=pd.get_dummies(input_data,columns=['id'],drop_first=True)

missing_columns=set(x_train.columns)-set(input_data_encoded.columns)
for col in missing_columns:
    input_data_encoded[col]=0
input_data_encoded=input_data_encoded[x_train.columns]

predicted_pollutants=model.predict(input_data_encoded)

print(f"\npredicted pollutant levels{station_id}in {year}")
for p,val in zip(pollutants,predicted_pollutants[0]):
    print(f"{p}:{val}")


predicted pollutant levels22in 2024
id:0.22632508682983693
date:4.982801134649662
NH4:19.45075441919192
BSK5:11.466240057165058
Suspended:6.981158333333334
O2:0.11372520256132755
NO3:387.6507135619936
NO2:0.26894651398601405
SO4:2182.7712223412705


In [46]:
import joblib

joblib.dump(model,'pollutionmodel.pkl')
joblib.dump(x_encoded.columns.tolist(), "modelcolumns.pkl")


['modelcolumns.pkl']